In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import os
import requests
import re        #a little bit of regex needed for processing responses
from collections import OrderedDict
import json      #this thing helped processing json objects manually
import pickle     #saves data in a python type in hard storage
from googleapiclient import errors  #I ended up not using the Google libs, but they exist
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
plt.style.use('ggplot')

In [4]:
with open('api_key.txt', 'r') as api_key_file:
        api_key = api_key_file.read().rstrip()
        
# with open('secret.txt') as f_sec:
#         APP_SECRET = f_sec.read().rstrip()

# Using the Youtube API

In [ ]:
#######################################################################
#These libraries are not needed for this project, but they provide   ##
#additional functionality for other Google API calls                 ##
#######################################################################
'''
!pip install google-api-python-client
!pip install google-auth-oauthlib google-auth-httplib2
'''

In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/script.projects']

SAMPLE_CODE = '''
function helloWorld() {
  console.log("Hello, world!");
}
'''.strip()

SAMPLE_MANIFEST = '''
{
  "timeZone": "America/New_York",
  "exceptionLogging": "CLOUD"
}
'''.strip()

def main():
    """Calls the Apps Script API.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('script', 'v1', credentials=creds)

    # Call the Apps Script API
    try:
        # Create a new project
        request = {'title': 'My Script'}
        response = service.projects().create(body=request).execute()

        # Upload two files to the project
        request = {
            'files': [{
                'name': 'hello',
                'type': 'SERVER_JS',
                'source': SAMPLE_CODE
            }, {
                'name': 'appsscript',
                'type': 'JSON',
                'source': SAMPLE_MANIFEST
            }]
        }
        response = service.projects().updateContent(
            body=request,
            scriptId=response['scriptId']).execute()
        print('https://script.google.com/d/' + response['scriptId'] + '/edit')
    except errors.HttpError as error:
        # The API encountered a problem.
        print(error.content)

In [ ]:
main()

In [ ]:
def api_request(page_token, country_code):
    # Builds the URL and requests the JSON from it
    request_url = f"https://www.googleapis.com/youtube/v3/videos?\
    part=id,statistics,snippet{page_token}chart=mostPopular&regionCode={country_code}&maxResults=100&key={api_key}"
    print("request_url:", request_url)
    return None
    request = requests.get(request_url)
    if request.status_code == 429:
        print("Temp-Banned due to excess requests, please wait and continue later")
        sys.exit()
    return request.json()

In [ ]:
page  = 'jTHWAxcMNk0'    #Ms Hintz's video of Google Classroom

In [5]:
def drill_down(nest, level=''):
    '''prints the structure out of a json object'''
    '''only needed for analyzing the response structure'''
    for n, a in enumerate(nest):        
        t = type(nest[a])
        print(level, n, a, type(nest[a]), ('of '+ str(type(nest[a][0])) if t == list else '') )
        if type(nest[a]) == list:
            drill_down(nest[a][0], level=level+'\t')
        elif type(nest[a]) == dict:
            drill_down(nest[a], level = level + '\t')

In [6]:
# First gather the categories and "guide" categories from the API
# This counts as a hit on the API, so only do this once and then save the file

cat_path  = 'data/categories.response.pickle'
categories_url = 'https://www.googleapis.com/youtube/v3/videoCategories?part=snippet&regionCode=US&key='

gcat_path = 'data/guide_categories.response.pickle'
guide_categories_url = 'https://www.googleapis.com/youtube/v3/guideCategories?part=snippet&regionCode=US&key='

#### Load user-given categories
if os.path.exists(cat_path):
    with open(cat_path, 'rb') as pickle_cat:
        categories_response = pickle.load(pickle_cat)
else:
    categories_response  = requests.request('GET', categories_url + api_key)
    with open(cat_path, 'wb') as cats:
        pickle.dump(categories_response, cats)

df = pd.DataFrame(categories_response.json()['items'])
categories = df.join(df.from_records(df['snippet']))


#### Load YouTube-assigned categories
if os.path.exists(gcat_path):
    with open(gcat_path, 'rb') as pickle_cat:
        guide_categories_response = pickle.load(pickle_cat)
else:
    guide_categories_response  = requests.request('GET', guide_categories_url + api_key)
    with open(gcat_path, 'wb') as cats:
        pickle.dump(guide_categories_response, cats)

df = pd.DataFrame(guide_categories_response.json()['items'])
guide_categories = df.join(df.from_records(df['snippet']))

In [8]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   kind        32 non-null     object
 1   etag        32 non-null     object
 2   id          32 non-null     object
 3   snippet     32 non-null     object
 4   channelId   32 non-null     object
 5   title       32 non-null     object
 6   assignable  32 non-null     bool  
dtypes: bool(1), object(6)
memory usage: 1.7+ KB


In [ ]:


drill_down( r_cat.json())
drill_down(r_gcat.json())

#put the categories into a pandas df
j = r_cat.json()
df = pd.DataFrame(j.json()['items'])


j = r_gcat.json()
df = pd.DataFrame(j['items'])
guide_categories = df.join(df.from_records(df['snippet']))


df7 = categories.loc[:,['id', 'title', 'assignable']]
df7.sort_values(['title'])

df8 = guide_categories[['id', 'title']]
df8.sort_values(['title'])


In [ ]:
url = '''https://www.googleapis.com/youtube/v3/videos?part=statistics&id='''
url += page
url += '&key='
angies_video = requests.request('GET', url + api_key)

In [ ]:
angies_video.json()['items'][0]['statistics']['viewCount']

In [ ]:
https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=Ks-_Mh1QhMc&key=[YOUR_API_KEY] HTTP/1.1


In [ ]:
print(api_key)

In [ ]:
fig, ax = plt.subplots()

In [ ]:
import requests, sys, time, os, argparse

# List of simple to collect features
snippet_features = ["title",
                    "publishedAt",
                    "channelId",
                    "channelTitle",
                    "categoryId"]

# Any characters to exclude, generally these are things that become problematic in CSV files
unsafe_characters = ['\n', '"']

# Used to identify columns, currently hardcoded order
header = ["video_id"] + snippet_features + ["trending_date", "tags", "view_count", "likes", "dislikes",
                                            "comment_count", "thumbnail_link", "comments_disabled",
                                            "ratings_disabled", "description"]


def setup(api_path, code_path):
    with open(api_path, 'r') as file:
        api_key = file.readline()

    with open(code_path) as file:
        country_codes = [x.rstrip() for x in file]

    return api_key, country_codes


def prepare_feature(feature):
    # Removes any character from the unsafe characters list and surrounds the whole item in quotes
    for ch in unsafe_characters:
        feature = str(feature).replace(ch, "")
    return f'"{feature}"'


def api_request(page_token, country_code):
    # Builds the URL and requests the JSON from it
    request_url = f"https://www.googleapis.com/youtube/v3/videos?part=id,statistics,snippet{page_token}chart=mostPopular&regionCode={country_code}&maxResults=50&key={api_key}"
    request = requests.get(request_url)
    if request.status_code == 429:
        print("Temp-Banned due to excess requests, please wait and continue later")
        sys.exit()
    return request.json()


def get_tags(tags_list):
    # Takes a list of tags, prepares each tag and joins them into a string by the pipe character
    return prepare_feature("|".join(tags_list))


def get_videos(items):
    lines = []
    for video in items:
        comments_disabled = False
        ratings_disabled = False

        # We can assume something is wrong with the video if it has no statistics, often this means it has been deleted
        # so we can just skip it
        if "statistics" not in video:
            continue

        # A full explanation of all of these features can be found on the GitHub page for this project
        video_id = prepare_feature(video['id'])

        # Snippet and statistics are sub-dicts of video, containing the most useful info
        snippet = video['snippet']
        statistics = video['statistics']

        # This list contains all of the features in snippet that are 1 deep and require no special processing
        features = [prepare_feature(snippet.get(feature, "")) for feature in snippet_features]

        # The following are special case features which require unique processing, or are not within the snippet dict
        description = snippet.get("description", "")
        thumbnail_link = snippet.get("thumbnails", dict()).get("default", dict()).get("url", "")
        trending_date = time.strftime("%y.%d.%m")
        tags = get_tags(snippet.get("tags", ["[none]"]))
        view_count = statistics.get("viewCount", 0)

        # This may be unclear, essentially the way the API works is that if a video has comments or ratings disabled
        # then it has no feature for it, thus if they don't exist in the statistics dict we know they are disabled
        if 'likeCount' in statistics and 'dislikeCount' in statistics:
            likes = statistics['likeCount']
            dislikes = statistics['dislikeCount']
        else:
            ratings_disabled = True
            likes = 0
            dislikes = 0

        if 'commentCount' in statistics:
            comment_count = statistics['commentCount']
        else:
            comments_disabled = True
            comment_count = 0

        # Compiles all of the various bits of info into one consistently formatted line
        line = [video_id] + features + [prepare_feature(x) for x in [trending_date, tags, view_count, likes, dislikes,
                                                                       comment_count, thumbnail_link, comments_disabled,
                                                                       ratings_disabled, description]]
        lines.append(",".join(line))
    return lines


def get_pages(country_code, next_page_token="&"):
    country_data = []

    # Because the API uses page tokens (which are literally just the same function of numbers everywhere) it is much
    # more inconvenient to iterate over pages, but that is what is done here.
    while next_page_token is not None:
        # A page of data i.e. a list of videos and all needed data
        video_data_page = api_request(next_page_token, country_code)

        # Get the next page token and build a string which can be injected into the request with it, unless it's None,
        # then let the whole thing be None so that the loop ends after this cycle
        next_page_token = video_data_page.get("nextPageToken", None)
        next_page_token = f"&pageToken={next_page_token}&" if next_page_token is not None else next_page_token

        # Get all of the items as a list and let get_videos return the needed features
        items = video_data_page.get('items', [])
        country_data += get_videos(items)

    return country_data

from collections import OrderedDict

def write_to_file(country_code, country_data):

    print(f"Writing {country_code} data to file...")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(f"{output_dir}/{time.strftime('%y.%d.%m.%H:%M')}_{country_code}_videos.csv", "w+", encoding='utf-8') as file:
        for row in country_data:
            file.write(f"{row}\n")


def get_data():
    for country_code in country_codes:
        country_data = [",".join(header)] + get_pages(country_code)
        write_to_file(country_code, country_data)


def pull_popular_videos():
    parser = argparse.ArgumentParser()
#     parser.add_argument('--key_path', help='Path to the file containing the api key, by default will use api_key.txt in the same directory', default='api_key.txt')
#     parser.add_argument('--country_code_path', help='Path to the file containing the list of country codes to scrape, by default will use country_codes.txt in the same directory', default='country_codes.txt')
#     parser.add_argument('--output_dir', help='Path to save the outputted files in', default='data/')

    args = parser.parse_args()

    output_dir = args.output_dir
    api_key, country_codes = setup(args.key_path, args.country_code_path)

    get_data()

In [ ]:
p = pull_popular_videos()

In [ ]:
#videos resource menu
#https://developers.google.com/youtube/v3/docs/videos

videos_menu = {
  "kind": "youtube#video",
  "etag": etag,
  "id": string,
  "snippet": {
    "publishedAt": datetime,
    "channelId": string,
    "title": string,
    "description": string,
    "thumbnails": {
      (key): {
        "url": string,
        "width": unsigned integer,
        "height": unsigned integer
      }
    },
    "channelTitle": string,
    "tags": [
      string
    ],
    "categoryId": string,
    "liveBroadcastContent": string,
    "defaultLanguage": string,
    "localized": {
      "title": string,
      "description": string
    },
    "defaultAudioLanguage": string
  },
  "contentDetails": {
    "duration": string,
    "dimension": string,
    "definition": string,
    "caption": string,
    "licensedContent": boolean,
    "regionRestriction": {
      "allowed": [
        string
      ],
      "blocked": [
        string
      ]
    },
    "contentRating": {
      "acbRating": string,
      "agcomRating": string,
      "anatelRating": string,
      "bbfcRating": string,
      "bfvcRating": string,
      "bmukkRating": string,
      "catvRating": string,
      "catvfrRating": string,
      "cbfcRating": string,
      "cccRating": string,
      "cceRating": string,
      "chfilmRating": string,
      "chvrsRating": string,
      "cicfRating": string,
      "cnaRating": string,
      "cncRating": string,
      "csaRating": string,
      "cscfRating": string,
      "czfilmRating": string,
      "djctqRating": string,
      "djctqRatingReasons": [,
        string
      ],
      "ecbmctRating": string,
      "eefilmRating": string,
      "egfilmRating": string,
      "eirinRating": string,
      "fcbmRating": string,
      "fcoRating": string,
      "fmocRating": string,
      "fpbRating": string,
      "fpbRatingReasons": [,
        string
      ],
      "fskRating": string,
      "grfilmRating": string,
      "icaaRating": string,
      "ifcoRating": string,
      "ilfilmRating": string,
      "incaaRating": string,
      "kfcbRating": string,
      "kijkwijzerRating": string,
      "kmrbRating": string,
      "lsfRating": string,
      "mccaaRating": string,
      "mccypRating": string,
      "mcstRating": string,
      "mdaRating": string,
      "medietilsynetRating": string,
      "mekuRating": string,
      "mibacRating": string,
      "mocRating": string,
      "moctwRating": string,
      "mpaaRating": string,
      "mpaatRating": string,
      "mtrcbRating": string,
      "nbcRating": string,
      "nbcplRating": string,
      "nfrcRating": string,
      "nfvcbRating": string,
      "nkclvRating": string,
      "oflcRating": string,
      "pefilmRating": string,
      "rcnofRating": string,
      "resorteviolenciaRating": string,
      "rtcRating": string,
      "rteRating": string,
      "russiaRating": string,
      "skfilmRating": string,
      "smaisRating": string,
      "smsaRating": string,
      "tvpgRating": string,
      "ytRating": string
    },
    "projection": string,
    "hasCustomThumbnail": boolean
  },
  "status": {
    "uploadStatus": string,
    "failureReason": string,
    "rejectionReason": string,
    "privacyStatus": string,
    "publishAt": datetime,
    "license": string,
    "embeddable": boolean,
    "publicStatsViewable": boolean,
    "madeForKids": boolean,
    "selfDeclaredMadeForKids": boolean
  },
  "statistics": {
    "viewCount": unsigned long,
    "likeCount": unsigned long,
    "dislikeCount": unsigned long,
    "favoriteCount": unsigned long,
    "commentCount": unsigned long
  },
  "player": {
    "embedHtml": string,
    "embedHeight": long,
    "embedWidth": long
  },
  "topicDetails": {
    "topicIds": [
      string
    ],
    "relevantTopicIds": [
      string
    ],
    "topicCategories": [
      string
    ]
  },
  "recordingDetails": {
    "recordingDate": datetime
  },
  "fileDetails": {
    "fileName": string,
    "fileSize": unsigned long,
    "fileType": string,
    "container": string,
    "videoStreams": [
      {
        "widthPixels": unsigned integer,
        "heightPixels": unsigned integer,
        "frameRateFps": double,
        "aspectRatio": double,
        "codec": string,
        "bitrateBps": unsigned long,
        "rotation": string,
        "vendor": string
      }
    ],
    "audioStreams": [
      {
        "channelCount": unsigned integer,
        "codec": string,
        "bitrateBps": unsigned long,
        "vendor": string
      }
    ],
    "durationMs": unsigned long,
    "bitrateBps": unsigned long,
    "creationTime": string
  },
  "processingDetails": {
    "processingStatus": string,
    "processingProgress": {
      "partsTotal": unsigned long,
      "partsProcessed": unsigned long,
      "timeLeftMs": unsigned long
    },
    "processingFailureReason": string,
    "fileDetailsAvailability": string,
    "processingIssuesAvailability": string,
    "tagSuggestionsAvailability": string,
    "editorSuggestionsAvailability": string,
    "thumbnailsAvailability": string
  },
  "suggestions": {
    "processingErrors": [
      string
    ],
    "processingWarnings": [
      string
    ],
    "processingHints": [
      string
    ],
    "tagSuggestions": [
      {
        "tag": string,
        "categoryRestricts": [
          string
        ]
      }
    ],
    "editorSuggestions": [
      string
    ]
  },
  "liveStreamingDetails": {
    "actualStartTime": datetime,
    "actualEndTime": datetime,
    "scheduledStartTime": datetime,
    "scheduledEndTime": datetime,
    "concurrentViewers": unsigned long,
    "activeLiveChatId": string
  },
  "localizations": {
    (key): {
      "title": string,
      "description": string
    }
  }
}

In [ ]:
#what parts do you want to request?
menu_selection = ['snippet', 'contentDetails', 'statistics']    #I just picked these from their menu
parts = '%2C'.join(menu_selection)   #join the parts together in a comma separated list, in url encoding as '%2C'
print(parts)
url = f'https://www.googleapis.com/youtube/v3/videos?part={parts}&id={page}&key={api_key}'
print(url)

In [ ]:
def pull_page_info(video_id, **kwargs):
    menu_selection = ['snippet', 'contentDetails', 'statistics']    #I just picked these from their menu
    parts = '%2C'.join(menu_selection)   #join the parts together in a comma separated list, in url encoding as '%2C'
    url = f'https://www.googleapis.com/youtube/v3/videos?part={parts}&id={video_id}&key={api_key}'
    return requests.request('GET', url)

In [ ]:
print('error' if new.status_code != 200 else 'ok')
new.status_code

In [ ]:
drill_down(new.json())

In [ ]:
chickensan_bad_guy = 'IncQIx_m_FM'
ten_hour_video = 'L_LUpnjgPso'

In [ ]:
import json
_ = json.loads(new.text)
short = _['items'][0]['contentDetails']['duration']
print(short)

In [ ]:
a = pull_page_info(ten_hour_video)

In [ ]:
long = json.loads(a.text)['items'][0]['contentDetails']['duration']
print(long)

In [ ]:
import re
    pattern = '[0-9]+[HMS]'
    result = re.findall(pattern, long)

In [ ]:
result

In [ ]:
nums = [int(val[:-1]) for val in result]
nums

In [ ]:
def convert_to_seconds(duration):
    import re
    pattern = '[0-9]+[HMS]'
    result = re.findall(pattern, duration)
    #result for 10 hours, 1 minute, 26 seconds = ['10H', '1M', '26S']
    
    nums = [int(val[:-1]) for val in result]
    
    if len(nums) == 1:
        nums = [0, 0].append(nums[0])
    elif len(nums) == 2:
        nums = [0] + nums
        print(nums)
        
    seconds = nums[0] * 60 * 60 + nums[1]*60 + nums[2]
    return seconds

In [ ]:
nums = [5,26]
n2 = [0] + nums
print(n2)

result = [3600, 60, 1] * nums
print(result)

In [ ]:
aba = convert_to_seconds(short)
aba


In [ ]:
import time
print (time.strftime('%y.%d.%m.%H.%M'))

In [ ]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.activities.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python

import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret_file.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube.activities().list(
        
    )
    response = request.execute()

    print(response)

if __name__ == "__main__":
    main()

In [ ]:
df = pd.read_csv('data/20.08.04.10.42_US_videos.csv')

In [ ]:
df.info()

In [ ]:
category

In [ ]:
df.head()

In [ ]:
print(df.sort_values('view_count')['view_count'])
df_sorted = df.sort_values('view_count')
df_sorted['view_count'].plot()
df.hist('view_count', bins=1000)
# plt.setlabel('Label')

In [ ]:
df['categoryId'].hist()